# This notebook test the difference of Gemini pro 001 and 002

In [6]:
from vertexai.generative_models import GenerationConfig, GenerativeModel, Content, Part  
# from google.cloud import aiplatform  

## Simple question answering

In [3]:
# 001
question='how many superbowls has Los Angeles Rams won?'
model = GenerativeModel("gemini-1.5-pro-001")
response = model.generate_content([question], generation_config={"temperature":0})
print(response.text)

The Los Angeles Rams have won **two** Super Bowls:

* **Super Bowl XXXIV (2000):** Defeated the Tennessee Titans 23-16
* **Super Bowl LVI (2022):** Defeated the Cincinnati Bengals 23-20 

It's worth noting that one of their Super Bowl victories (XXXIV) was when the team was based in St. Louis. 



<font color='green'><b>correct</b></font>

In [2]:
# 002
question='how many superbowls has Los Angeles Rams won?'
model = GenerativeModel("gemini-1.5-pro-002")
response = model.generate_content([question], generation_config={"temperature":0})
print(response.text)

The Los Angeles Rams have won **three** Super Bowls:

* **Super Bowl XXXIV (2000):** Defeated the Tennessee Titans
* **Super Bowl LVI (2022):** Defeated the Cincinnati Bengals
* **Super Bowl XIV (1980) when they were located in Los Angeles** Defeated the Pittsburgh Steelers.  They also won one championship prior to the Super Bowl era (1951) when they were located in Cleveland.



<font color='red'><b>incorrect, the winner is Pittsburgh in 1980, the source of Gemini-pro-002 is in doubt</b></font> 

In [5]:
# 002
question='how many superbowls has Los Angeles Rams won, and what are the score?'
model = GenerativeModel("gemini-1.5-pro-002")
response = model.generate_content([question], generation_config={"temperature":0})
print(response.text)

The Los Angeles Rams have won **two** Super Bowls since becoming the Los Angeles Rams (they also won one as the St. Louis Rams):

* **Super Bowl XXXIV (2000):** St. Louis Rams 23, Tennessee Titans 16
* **Super Bowl LVI (2022):** Los Angeles Rams 23, Cincinnati Bengals 20



<font color='green'><b>correct, but only with proper guidance</b></font>

In [39]:
# 001
question="What is the Well Fargo's net income in 2023"
model = GenerativeModel("gemini-1.5-pro-001")
response = model.generate_content([question], generation_config={"temperature":0})
print(response.text)

I do not have access to real-time information, including financial data like Wells Fargo's net income. 

To find this information, I recommend checking these resources:

* **Wells Fargo's Investor Relations Website:** This is the most reliable source for official financial data. Look for their quarterly earnings reports (usually labeled as 10-Q) or annual reports (10-K).
* **Financial News Websites:** Websites like Yahoo Finance, Google Finance, Bloomberg, and Reuters will publish financial results for publicly traded companies like Wells Fargo shortly after they are announced.
* **SEC EDGAR Database:** You can find official company filings, including financial statements, on the U.S. Securities and Exchange Commission's EDGAR database.

Please note that Wells Fargo, like most companies, releases financial results on a quarterly and annual basis. So, you'll likely find the most up-to-date information a few weeks after each quarter ends. 



<font color='red'><b>do not have real-time info</b></font> 

In [40]:
# 002
question="What is the Well Fargo's net income in 2023"
model = GenerativeModel("gemini-1.5-pro-002")
response = model.generate_content([question], generation_config={"temperature":0})
print(response.text)

Wells Fargo's full-year 2023 net income was **$14.431 billion**.



<font color='red'><b>incorrect. In 2023, Wells Fargo generated $19.1 billion in net income</b></font> 

* 002 has more online info retrieval, but the source info might be incorrect

## Financial question answering using one big prompt

In [8]:
import sys
import os
import importlib
sys.path.append(os.path.abspath('../utils'))
import big_prompt_agent
importlib.reload(big_prompt_agent)
from big_prompt_agent import BigPromptAgent

In [13]:
agent_001=BigPromptAgent(pdf_path='../data/naf23.pdf', model='gemini-1.5-pro-001')
query="what is National Ataxia Foundation's total revenue in 2023?"

answer=agent_001.run(query=query)
print(answer)

The provided auditor's report does not contain the information needed to answer your question. Therefore, I cannot answer your question. 


<font color='red'><b>incorrect, cannot find the answer</b></font> 

In [11]:
agent_002=BigPromptAgent(pdf_path='../data/naf23.pdf', model='gemini-1.5-pro-002')
query="what is National Ataxia Foundation's total revenue in 2023?"

answer=agent_002.run(query=query)
print(answer)

The provided financial statements are for the years ended December 31, 2023 *and* 2022.  The Statement of Activities shows the following total support and revenue for 2023:

* **Total Support:** $3,205,136
* **Total Revenue:** $979,651
* **Net Assets Released from Restrictions:** ($975,954) - Note that this is a *decrease* in temporarily restricted net assets.
* **Total Support and Revenue:** $4,184,787


<font color='green'><b>correct</b></font>

In [14]:
query="what is National Ataxia Foundation's expense made of "
answer=agent_001.run(query=query)
print(answer)

The provided document is an annual report of National Ataxia Foundation. It does not contain the information about the makeup of the expense. 



<font color='red'><b>incorrect, cannot find the answer</b></font> 

In [15]:
query="what is National Ataxia Foundation's expense made of "
answer=agent_002.run(query=query)
print(answer)

The Statement of Functional Expenses on page 9 of the provided financial report shows the breakdown of the National Ataxia Foundation's expenses for the year ended December 31, 2023.  The expenses are categorized as Program Services and Supporting Services.

**Program Services:**

* **Research:** $1,574,354
* **Education and Service:** $1,146,980
* **Drug Development Collaborative:** $1,188,427
* **Total Program Services:** $3,909,761

**Supporting Services:**

* **Management and General:** $538,079
* **Fundraising:** $548,618
* **Total Supporting Services:** $1,086,697

**Total Expenses:** $4,996,458


Further details on the composition of each expense category (salaries, payroll taxes, fringe benefits, bank fees, etc.) can be found within the Statement of Functional Expenses itself.


<font color='green'><b>correct</b></font>

In [16]:
query="what does National Ataxia Foundation do "
answer=agent_001.run(query=query)
print(answer)

The provided document is an annual financial report of the National Ataxia Foundation. There is no information about what the foundation does in the provided excerpt. 



In [17]:
query="what do you know about National Ataxia Foudnation"
answer=agent_001.run(query=query)
print(answer)

The provided document is the annual financial report of the National Ataxia Foundation. It does not contain information about the foundation itself, but rather its financial position as of December 31, 2023, and 2022. 



<font color='red'><b>incorrect, couldn't answer simple questions</b></font> 

## ReAct with RAG

In [18]:
from IPython.display import Markdown,display,HTML


import sys
import json
import os
import importlib
import math
import html

sys.path.append(os.path.abspath('../utils'))

import tool_functions
importlib.reload(tool_functions)
from tool_functions import convert_to_tool # convert self-defined functions to Tool objects

import rag
importlib.reload(rag)
from rag import RAG # RAG search function

import react_agent
importlib.reload(react_agent)
from react_agent import ReactAgent

In [19]:
# read the data, create an RAG class on that data, create the search function
data_path='../data/naf23.pdf'

from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# create a faiss_vector_store
pdf_reader = PdfReader(data_path)
text = ""
chunks=[]

# read the pages
for page in pdf_reader.pages:
    text += page.extract_text()

# text_splitter.create_documents() splits the text into chunks, then change each chunk into a 'Document' object of Langchain
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
chunks=text_splitter.create_documents([text])

# embedding model
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

# faiss_vector_store
faiss_vector_store = FAISS.from_documents(chunks, embedding_model)

rag_instance=RAG(pdf_path= data_path, chunking_method='recursive', faiss_vector_store=faiss_vector_store)

def search(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    results = rag_instance.search(query, method='ensemble')
    return json.dumps({"relevant information in the auditor notes": results})

# convert this search function to a Tool object
search_tool = convert_to_tool(search)

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
agent_react_001 = ReactAgent(tools=[search_tool],model='gemini-1.5-pro-001')
agent_react_002 = ReactAgent(tools=[search_tool],model='gemini-1.5-pro-002')


### question 1

In [22]:
query = "what is the total revenue of National Ataxia Foundation in 2023?"


In [23]:
agent_react_001.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "Revenue\nConference income 264,816           -                        264,816           \nEarned income 570,833           -                        570,833           \nInvestment income 144,002           -                        144,002           \nTotal Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079     

'The total revenue of National Ataxia Foundation in 2023 is 979,651.'

In [24]:
agent_react_002.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "Revenue\nConference income 264,816           -                        264,816           \nEarned income 570,833           -                        570,833           \nInvestment income 144,002           -                        144,002           \nTotal Revenue 979,651           -                        979,651           \nNet Assets Released from Restrictions 975,954           (975,954)          -                        \nTotal Support and Revenue 3,546,026       638,761           4,184,787       \nExpenses\nProgram services\nResearch 1,574,354       -                        1,574,354       \nEducation and service 1,146,980       -                        1,146,980       \nDrug Development Collaborative 1,188,427       -                        1,188,427       \nTotal Program Services 3,909,761       -                        3,909,761       \nSupporting services\nManagement and general 538,079           -                        538,079     

'The total revenue for National Ataxia Foundation in 2023 is $979,651.'

* both are correct, since this is a basic retrieval question
* with the help of RAG, even 001 can answer it

### question 2

In [25]:
query="What caused the significant change in National Ataxia Foundation's net assets with donor restrictions between 2022 and 2023?"

In [26]:
agent_react_001.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "and stored at the NINDS SCA -BRAC biorepository. Both de-identified clinical data and biofluids are available \nfor request from non -participating investigators for approved research projects.  \n  \n13 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nB. Basis of Accounting and Presentation  \n \nThe accompanying financial statements have been prepared using  the accrual basis of accounting in accordance with \naccounting principles generally accepted in the United States of America.   \n \nRevenues are recorded when earned and expenses are recorded when a liability is incurred. Contributions received are \nrecorded as an increase in non -donor -restricted or donor -restricted support depending on the existence or nature of any \ndonor restrictions.  Accordingly, net assets of the Foundation and changes therein are 

"I cannot answer your question based on the provided information. The auditor notes do not explain the reason for the change in National Ataxia Foundation's net assets with donor restrictions between 2022 and 2023."

In [27]:
agent_react_002.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "and stored at the NINDS SCA -BRAC biorepository. Both de-identified clinical data and biofluids are available \nfor request from non -participating investigators for approved research projects.  \n  \n13 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nB. Basis of Accounting and Presentation  \n \nThe accompanying financial statements have been prepared using  the accrual basis of accounting in accordance with \naccounting principles generally accepted in the United States of America.   \n \nRevenues are recorded when earned and expenses are recorded when a liability is incurred. Contributions received are \nrecorded as an increase in non -donor -restricted or donor -restricted support depending on the existence or nature of any \ndonor restrictions.  Accordingly, net assets of the Foundation and changes therein are 

{"relevant information in the auditor notes": "121,917 $        - $                     Programs and \nFundraising\nWebsite Services -                        9,870               Management and \nGeneralEstimated wholesale price of \nidentical or similar products\nTotal In-kind Contributions 121,917 $        9,870 $            Google and                         \nMicrosoft Advertising GrantsEstimated wholesale price of \nidentical or similar products\n \nThe in -kind contributions as of December 31, 2023  and 2022  had no donor restrictions.  \n  \n20 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 10: Liquidity and Availability of Financial Assets  \n \nFinancial assets available for general expenditure, that is, without donor or other restrictions limiting their use, within o ne \nyear of the statement of financial position dates, comprise the following:  \n \n2023 2022\nCash and cash equivalents 1,008,716$     1,969,164$   

{"relevant information in the auditor notes": "Annual Financial  \nReport  \nNational Ataxia Foundation   \nSt. Louis Park, Minnesota  \n \n \nFor the years ended December 31, 2023  and 2022  \n National Ataxia Foundation  \nTable of Contents  \nDecember 31, 2023  and 2022  \nPage No.  \nIndependent Auditor's Report  3 \nFinancial Statements  \nStatement s of Financial Position  6 \nStatement s of Activities  7 \nStatements of Functional Expenses  9 \nStateme nts of Cash Flows  11 \nNotes to the Financial Statements  12 \n2 \n  \n \n \n \n \n \nINDEPENDENT AUDITOR'S REPORT  \n \n \nBoard of Directors  \nNational Ataxia Foundation  \nSt. Louis Park , Minnesota  \n \nOpinion  \n \nWe have audited the accompanying financial statements of National Ataxia Foundation  (the Foundation ), which comprise \nthe statements of financial position as of December 31, 2023  and 2022 , and the related statements of activities, \nfunctional expenses and cash flows for the years then ended, and the relat

"I'm unable to pinpoint the precise cause of the change in the National Ataxia Foundation's net assets with donor restrictions between 2022 and 2023 based on the information I could find.  The financial statements show an increase, but don't detail the specific transactions or events that led to it.  Further investigation, perhaps reviewing the full audit report or contacting the foundation directly, would be necessary to determine the exact reasons."

* 001 gives up in just one iteration
* 002 can find the change in numbers, but could not find the drivers
* the drivers is not retrieved, `"such as the SCA2 Fund and MR Imaging Study Fund, as outlined in the notes on donor-restricted assets."`

### question 3

In [28]:
query="What was National Ataxia Foundation's operating lease liability at the end of 2023, and how does the lease structure affect NAF’s financial obligations?"

In [29]:
agent_react_001.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "Annual Financial  \nReport  \nNational Ataxia Foundation   \nSt. Louis Park, Minnesota  \n \n \nFor the years ended December 31, 2023  and 2022  \n National Ataxia Foundation  \nTable of Contents  \nDecember 31, 2023  and 2022  \nPage No.  \nIndependent Auditor's Report  3 \nFinancial Statements  \nStatement s of Financial Position  6 \nStatement s of Activities  7 \nStatements of Functional Expenses  9 \nStateme nts of Cash Flows  11 \nNotes to the Financial Statements  12 \n2 \n  \n \n \n \n \n \nINDEPENDENT AUDITOR'S REPORT  \n \n \nBoard of Directors  \nNational Ataxia Foundation  \nSt. Louis Park , Minnesota  \n \nOpinion  \n \nWe have audited the accompanying financial statements of National Ataxia Foundation  (the Foundation ), which comprise \nthe statements of financial position as of December 31, 2023  and 2022 , and the related statements of activities, \nfunctional expenses and cash flows for the years then ended, and the relat

{"relevant information in the auditor notes": "included in long -term liabilities and current liabilities on the statement of financial position. ROU assets are valued at the \ninitial measurement of the lease liability, plus any  indirect costs or rent prepayments, and reduced by any lease incentives \nand any deferred lease payments. Operating ROU assets are recorded on the face of the statement of financial position \nand are amortized over the lease term. To determine the present v alue of lease payments on lease commencement, the \nFoundation  uses the implicit rate when readily determinable. Lease terms include options to extend or terminate the lease \nwhen it is reasonably certain that the Foundation  will exercise that option. Lease expense is recognized on a straight -line \nbasis over the life of the lease and is included within operating expenses on the statement of activities.  \nThe Foundation  has made the following elections related to leases:  \n \n\u2022 The Foundatio

"The National Ataxia Foundation's operating lease liability was $58,103 at the end of 2023. \n\nThe lease structure, likely an operating lease, impacts NAF's financial obligations in that it spreads the cost of the lease over the lease term. This means NAF makes regular lease payments, impacting their short-term cash flow. However, these payments are generally smaller than if they had purchased the asset outright. \n\nIt's important to note that I cannot provide a complete analysis without more context regarding the specific lease agreement and NAF's overall financial picture."

In [30]:
agent_react_002.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "according to the Foundation 's elected policy. The Foundation 's lease agreement does not contain any material residual \nvalue guarantees or material restrictive covenants.  \n \nAdditional information about the Foundation \u2019s lease for the year ended December 31, 2023 , is as follows:  \n \nLease expense (included in operating expenses)\nOperating lease expense 33,406 $          \nVariable lease expense 27,997             \nTotal Lease Expense: 61,403 $          \nOther Information\nCash paid for amounts included in the measurement of lease liabilities\nOperating cash flows from operating leases 33,240 $          \nWeighted-average remaining lease term in years for operating leases 1.75                 \nWeighted-average discount rate for operating leases 2.333%\nFuture minimum payments for leases are as follows:\nYear Ended December 31, Amount\n2024 33,693 $          \n2025 25,552             \nTotal undiscounted cash flows 59,245  

"The National Ataxia Foundation's operating lease liability at the end of 2023 was $58,103.  The lease structure, with fixed monthly base payments increasing over time and a responsibility for a proportional share of property taxes and operating expenses, creates a predictable stream of financial obligations for the NAF.  The variable lease payments, not included in the liability, add some uncertainty to their expenses.  The lease expires in September 2025, meaning NAF has a commitment for these payments until that date."

* both 001 and 002 find the operating lease liability numbers
* 002 find the reason because it has a better search query that retrives the correct information.

## question 3

In [31]:
query ="How did National Ataxia Foundation's deferred revenue change between 2022 and 2023, and what drove this change?"

In [32]:
agent_react_001.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "the years ended December 31, 2023  and 2022 , respectively. Deferred Drug D evelopment Collaborative revenue \nwas $100,000  and $0 for the years ended December 31, 2023  and 2022 , respectively.  \n \nThe Foundation \u2019s other revenues are explicitly excluded from the scope of ASC Topic 606 and are not recorded in \naccordance with that standard.  All contributions are considered to be available for unrestricted use unless specifically \nrestricted by the donor. The Board\u2019s general policy is to allocate up to thirty percent of all restricted contributions for \nunrestricted use unless the contribution was solicited by the Foundation for a sp ecific purpose.  The board has discretion \nto use this allocation or waive it.  \n15 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nPr

"National Ataxia Foundation's deferred revenue increased from $0 in 2022 to $100,000 in 2023. This is due to the implementation of Accounting Standards Codification 606, which requires revenue from the Drug Development Collaborative to be recognized over the collaboration period rather than at a point in time."

In [33]:
agent_react_002.run(user_msg=query)

---------------------------------------- iteration 0  ----------------------------------------


{"relevant information in the auditor notes": "the years ended December 31, 2023  and 2022 , respectively. Deferred Drug D evelopment Collaborative revenue \nwas $100,000  and $0 for the years ended December 31, 2023  and 2022 , respectively.  \n \nThe Foundation \u2019s other revenues are explicitly excluded from the scope of ASC Topic 606 and are not recorded in \naccordance with that standard.  All contributions are considered to be available for unrestricted use unless specifically \nrestricted by the donor. The Board\u2019s general policy is to allocate up to thirty percent of all restricted contributions for \nunrestricted use unless the contribution was solicited by the Foundation for a sp ecific purpose.  The board has discretion \nto use this allocation or waive it.  \n15 \n National Ataxia Foundation  \nNotes to the Financial Statements  \nDecember 31, 2023  and 2022  \n \nNote 1:  Summary of Significant Accounting Policies (Continued)  \n \nJ. Property and Equipment  \n \nPr

"The National Ataxia Foundation's deferred revenue increased from $0 in 2022 to $100,000 in 2023. This change is attributed to $100,000 in revenue received in advance related to Drug Development Collaborative agreements."

* both 001 and 002 got the driver correct, because RAG retrieves the correct notes
* none of them got the number right, because RAG does not retrieve the statement that contain the number.
* 002 search 3 query in parallel, if we have advanced method handling (i.e., if we can provide a direct and correct answer to questions like 'National Ataxia Foundation deferred revenue 2023'), it could potentially saves the number of calls to LLM.

# Conclusion

* 002 is better than 001 in reasoning under the ReAct framework
* 002 might have some incorrect online information on public knowledge, a note of caution 
* under ReAct, the ability of RAG still matters a lot